In [0]:
import tensorflow as tf
import shutil # copy, move file
import os # miscellaneous operation system interfaces
import pathlib
import random

In [0]:
!python3 -m pip install tensorflow==2.0.0

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

In [0]:
import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_root = pathlib.Path(data_root_orig)

In [0]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

In [0]:
!rm -rf ./data/train/sunflowers ./data/train/tulips ./data/train/daisy ./data/train/dandelion ./data/train/roses  ./data/test/sunflowers ./data/test/tulips ./data/test/daisy ./data/test/dandelion ./data/test/roses

In [0]:
!mkdir ./data ./data/raw ./data/train ./data/test 
!mkdir ./data/train/sunflowers ./data/train/tulips ./data/train/daisy ./data/train/dandelion ./data/train/roses  ./data/test/sunflowers ./data/test/tulips ./data/test/daisy ./data/test/dandelion ./data/test/roses

In [0]:
!mv /root/.keras/datasets/flower_photos/* /content/data/raw

In [0]:
!rm /content/data/raw/LICENSE.txt

In [0]:
DATA_RAW_FOLDER = "./data/raw"
DATA_TRAIN_FOLDER = "./data/train"
DATA_TEST_FOLDER = "./data/test"

In [0]:
!ls data/raw/*/* -1q | wc -l

In [0]:
list_flowers = ['daisy', 'roses', 'sunflowers', 'dandelion', 'tulips']
list_data = []
sum = 0
for name in list_flowers:
  list_data.append(pathlib.Path(os.path.join(DATA_RAW_FOLDER, name)))

list_image = {'daisy': [],
              'roses': [], 
              'sunflowers': [], 
              'dandelion': [], 
              'tulips': []}

num_list = len(list_flowers)

for i in range(num_list):
  for item in list_data[i].glob('*'):
    name = str(item).split('/')[-2]
    list_image.get(name).append(item)
    sum += 1    

assert sum == 3670
n_sample = 1000
test_size = 0.2

for key, values in list_image.items():
  test_sample = random.sample(values, n_sample//5)
  for i in range(n_sample//5):
    if i < (n_sample * test_size // 5):
      shutil.copy(test_sample[i], os.path.join(DATA_TEST_FOLDER, key))
    else:
      shutil.copy(test_sample[i], os.path.join(DATA_TRAIN_FOLDER, key))

In [0]:
!ls -1q ./data/train/*/* | wc -l
!ls -1q ./data/test/*/* | wc -l

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:

TRAIN_DATA_DIR = './data/train/'
VALIDATION_DATA_DIR = './data/test/'
TRAIN_SAMPLES = 800
VALIDATION_SAMPLES = 200
NUM_CLASSES = 5
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [0]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [0]:
train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=102,
                        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                        VALIDATION_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        class_mode='categorical')

In [0]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    
    return Model(inputs=input, outputs=predictions)

In [0]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)              
model.fit_generator(train_generator,
          steps_per_epoch = num_steps,
          epochs=10,
          validation_data = validation_generator,
          validation_steps = num_steps)